In [1]:
import requests
from bs4 import BeautifulSoup
from simplekml import Kml
from IPython.display import FileLink

In [2]:
# URL of the ghost towns in  - change for diff state
URL = "https://en.wikipedia.org/wiki/List_of_ghost_towns_in_Colorado"
response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")

towns = []

for geo in soup.find_all("span", class_="geo"):
    try:
        lat, lon = map(float, geo.text.split(";"))
    except ValueError:
        continue

    row = geo.find_parent("tr")
    if row:
        cells = row.find_all("td")

        # make sure youre pulling the actual town name, even if inside a hyperlink
        name_tag = row.find("a")
        name = name_tag.get_text(strip=True) if name_tag else (cells[0].get_text(strip=True) if cells else "Unnamed Town")

        # Status cell - on the CO page it's cell 7 but change this if you're using a different page.
        status = cells[7].get_text(strip=True) if len(cells) > 7 else "Unknown"
    else:
        name = "Unnamed Town"
        status = "Unknown"

    towns.append((name, lat, lon, status))

print(f"Found {len(towns)} ghost towns with coordinates and status.") # so we know how many we got!

Found 200 ghost towns with coordinates and status.


In [3]:
#Create KML file
kml = Kml()
for name, lat, lon, status in towns:
    pnt = kml.newpoint(name=name, coords=[(lon, lat)])
    pnt.description = f"Status: {status}"

# Save to a temporary path for later downloadd
output_path = "/tmp/ghost_towns.kml"
kml.save(output_path)
print(f"KML saved to: {output_path}")

# click this output link to open the KML file, if necessary you can cahnge this to just print the actual KML contents then copy pastee into a text file (but still save it as .kml when you're done)
FileLink(output_path)

KML saved to: /tmp/ghost_towns.kml


/tmp/ghost_towns.kml